In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root");en="550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

# #546eV - 8.2eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root");en="546eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_10800-2350ns_multiCH11.root")

# 544eV - 6.1eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root");en="544eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH11.root")

# #542eV - 4eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root");en="542eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_10800-2350ns_COMBI_multiCH11.root")

# #541eV 3eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root");en="541eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_10800-2350ns_multiCH11.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew","MFPADs_multinew_phi"]

loc=path+IDs[0]+dirs[0]

#to alow the helicity comparison without the S enantiomer
if en != "550eV":
    fileSCR = None
    fileSCL = None

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

In [ ]:
#Loading of the MFPADS and cos len = 27
MFPAD_RCR,ctheta_RCR,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL=fugi.import_MFPAD(fileRCL, loc)
#projection of cos on x axis: len = 18
cosx_RCR=fugi.projection(MFPAD_RCR,alongaxis=0)
cosx_RCL=fugi.projection(MFPAD_RCL,alongaxis=0)

if fileSCL and fileSCR is not None:    
    MFPAD_SCR,ctheta_SCR=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL=fugi.import_MFPAD(fileSCL, loc)
    #projection of cos on x axis
    cosx_SCR=fugi.projection(MFPAD_SCR,alongaxis=0)
    cosx_SCL=fugi.projection(MFPAD_SCL,alongaxis=0)

In [ ]:
#import 3d graph kilian
MFPAD_RCLtest=fugi.import_MFPAD3D(fileRCL, loc)
MFPAD_RCRtest=fugi.import_MFPAD3D(fileRCR, loc)

#sum
MFPAD_RCLtest_sum=np.sum(MFPAD_RCLtest,axis=0)
MFPAD_RCRtest_sum=np.sum(MFPAD_RCRtest,axis=0)

#normalise each and sum
# MFPAD_RCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCLtest),axis=0)
# MFPAD_RCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCRtest),axis=0)
MFPAD_Rtest_sum = np.add(MFPAD_RCLtest_sum,MFPAD_RCRtest_sum)

if fileSCL and fileSCR is not None:
    MFPAD_SCRtest=fugi.import_MFPAD3D(fileSCR, loc)
    MFPAD_SCLtest=fugi.import_MFPAD3D(fileSCL, loc)
    MFPAD_SCRtest_sum=np.sum(MFPAD_SCRtest,axis=0)
    MFPAD_SCLtest_sum=np.sum(MFPAD_SCLtest,axis=0)

    # MFPAD_SCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCLtest),axis=0)
    # MFPAD_SCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCRtest),axis=0)

    #flip along cos(theta)
    MFPAD_SCLtest_flip_sum=np.flip(MFPAD_SCLtest_sum,axis=0)
    MFPAD_SCRtest_flip_sum=np.flip(MFPAD_SCRtest_sum,axis=0)
    MFPAD_Stest_sum = np.add(MFPAD_SCLtest_flip_sum,MFPAD_SCRtest_flip_sum)

    MFPAD_tottest=np.add(MFPAD_Stest_sum,MFPAD_Rtest_sum)

    print(MFPAD_tottest.shape)
else:
    MFPAD_tottest=MFPAD_Rtest_sum

In [ ]:
##########################b1 map x,y coordiantes###############################
"""
PHI - CTHETA photon with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)

xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, reduced=False, source=True)

# list(zip(xgo_phi,ygo_phi))
print("(xgo_cos (phi), ygo_cos (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("\n cosphi_adj_cos  \n", cosphi_adj_cos[:6])
print("\n cosphi_adj_phi \n", cosphi_adj_phi[:6])

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (36,), MFPAD_axis = (37,)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])

In [ ]:
#normalising entries, important just for PECD
MFPAD_RCR_norm = fugi.normalise_matrix(MFPAD_RCR,normtype=2) # z values normalise to number of event
MFPAD_RCL_norm = fugi.normalise_matrix(MFPAD_RCL,normtype=2) # z values normalise to number of event
cosx_RCR_norm = fugi.normalise_matrix(cosx_RCR,normtype=2) # z values normalise to number of event
cosx_RCL_norm = fugi.normalise_matrix(cosx_RCL,normtype=2) # z values normalise to number of event

if fileSCL and fileSCR is not None:
    MFPAD_SCR_norm = fugi.normalise_matrix(MFPAD_SCR,normtype=2) # z values normalise to number of event
    MFPAD_SCL_norm = fugi.normalise_matrix(MFPAD_SCL,normtype=2) # z values normalise to number of event
    cosx_SCR_norm = fugi.normalise_matrix(cosx_SCR,normtype=2) # z values normalise to number of event
    cosx_SCL_norm = fugi.normalise_matrix(cosx_SCL,normtype=2) # z values normalise to number of event

In [ ]:
MFPAD_RCR.shape

In [ ]:
#flipping; axis=1 flips along cos(theta), axis=0 along phi
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCL:
    MFPAD_RCL_flip.append(np.flip(el, axis=1))
MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCR:
    MFPAD_RCR_flip.append(np.flip(el, axis=1))
MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)

if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for el in MFPAD_SCL:
        MFPAD_SCL_flip.append(np.flip(el, axis=1))
    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)

    for el in MFPAD_SCR:
        MFPAD_SCR_flip.append(np.flip(el, axis=1))
    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)    

In [ ]:
#sum within an experimemt axis=0 is among the 72 elements
#single enantiomer and helicity
#sum all MFPAD: NO NORMALIZATION
MFPAD_RCR_sum=np.sum(MFPAD_RCR, axis=0)
MFPAD_RCL_sum=np.sum(MFPAD_RCL, axis=0)
MFPAD_R_sum=np.add(MFPAD_RCR_sum,MFPAD_RCL_sum)

if fileSCL and fileSCR is not None:
    #single enantiomer and helicity
    MFPAD_SCR_sum=np.sum(MFPAD_SCR, axis=0)
    MFPAD_SCL_sum=np.sum(MFPAD_SCL, axis=0)
    MFPAD_SCR_flip_sum=np.sum(MFPAD_SCR_flip, axis=0)
    MFPAD_SCL_flip_sum=np.sum(MFPAD_SCL_flip, axis=0)

    MFPAD_S_sum=np.add(MFPAD_SCR_flip_sum,MFPAD_SCL_flip_sum)

    #sum of two enantiomers: there is no influence of the ellicity
    MFPAD_tot_sumt=MFPAD_RCR_sum+MFPAD_RCL_sum+MFPAD_SCR_flip_sum+MFPAD_SCL_flip_sum #bug?
    MFPAD_tot_sum=np.add(MFPAD_R_sum,MFPAD_S_sum)
else:
    MFPAD_tot_sum=MFPAD_R_sum

In [ ]:
print(MFPAD_RCR.shape)
print(MFPAD_RCR.max(),MFPAD_RCR.min())

In [ ]:
#sorting
#let´s work just with NOT NORMALISED becuase of MFPAD
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, theory=False, a=1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, theory=False, a=1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)

MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=2)
MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=2)


if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, theory=False, a=1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, theory=False, a=1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)

    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=2)
    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=2)

In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in cos, do the phi just as a check
MFPAD_RCLCR=[]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            MFPAD_RCLCR.append(np.add(MFPAD_RCL_phi.reshape(6,12,-1)[i,j],MFPAD_RCR_phi.reshape(6,12,-1)[5-i,j-6]))
            i+=1
        else:
            MFPAD_RCLCR.append(np.add(MFPAD_RCL_phi.reshape(6,12,-1)[i,j],MFPAD_RCR_phi.reshape(6,12,-1)[5-i,6+j]))
            i+=1
MFPAD_RCLCR=np.array(MFPAD_RCLCR).reshape(72,MFPAD_RCL_phi.shape[1],MFPAD_RCL_phi.shape[2])
MFPAD_RCLCR_sum=np.sum(MFPAD_RCLCR, axis=0)

#sorting
MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!

if fileSCL and fileSCR is not None:
    MFPAD_SCLCR=[]
    MFPAD_totordered=[]
    #sum helicities: -cos(theta), phi+180 in photn coordiantes
    for j in range(12):
        for i in range(6):
            if j>5:
                MFPAD_SCLCR.append(np.add(MFPAD_SCL_phi.reshape(6,12,-1)[i,j],MFPAD_SCR_phi.reshape(6,12,-1)[5-i,j-6]))
                i+=1
            else:
                MFPAD_SCLCR.append(np.add(MFPAD_SCL_phi.reshape(6,12,-1)[i,j],MFPAD_SCR_phi.reshape(6,12,-1)[5-i,6+j]))
                i+=1
    MFPAD_SCLCR=np.array(MFPAD_SCLCR).reshape(72,MFPAD_SCL_phi.shape[1],MFPAD_SCL_phi.shape[2])
    MFPAD_SCLCR_sum=np.sum(MFPAD_SCLCR, axis=0)

    #sorting
    MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a= 1)
    MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a= 2) #equal to the original!
    
    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates
    nphi=MFPAD_SCL.shape[1];ncos=MFPAD_SCL.shape[2]
    for j in range(12):
        for i in range(6):
            MFPAD_totordered.append(np.add(MFPAD_SCLCR_phi.reshape(6,12,nphi,ncos)[i,j],np.flip(MFPAD_RCLCR_phi.reshape(6,12,nphi,ncos)[5-i,j], axis=1)))
            i+=1

    MFPAD_totordered=np.array(MFPAD_totordered).reshape(72,MFPAD_SCL.shape[1],MFPAD_SCL.shape[2])
    
    #sorting
    MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, theory=False, a=1)
    MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, theory=False, a=2) #equal to the original!

    #PECD check it is equal to contrast
    mPECD = np.divide(np.subtract(MFPAD_RCLCR_sum,MFPAD_SCLCR_sum),np.add(MFPAD_RCLCR_sum,MFPAD_SCLCR_sum))

else:
    MFPAD_totordered=MFPAD_RCLCR
    MFPAD_totordered_cos=MFPAD_RCLCR_cos
    MFPAD_totordered_phi=MFPAD_RCLCR_phi
    mPECD=np.divide(np.subtract(MFPAD_RCL_sum,MFPAD_RCR_sum),np.add(MFPAD_RCL_sum,MFPAD_RCR_sum))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,4))
ax1.contourf(phiM,cosM,MFPAD_RCR[0].T)
ax2.contourf(phiM,cosM,MFPAD_RCR_flip[0].T)
ax3.contourf(phiM,cosM,MFPAD_tottest)

# MFPAD_SCL_flip_cos[45]==np.flip(np.flip(MFPAD_SCL_cos[45],axis=1),axis=0) #True

In [ ]:
MFPAD_tottest
np.sum(MFPAD_RCRtest)

#test samrt sum
counts=0
for j in range(12):
    for i in range(6):
        if j>5:  
            print("b=",i,j,"a1=",5-i,j-6)
            i+=1
        else:
            print("b=",i,j,"a2=",5-i,j+6)
            i+=1
        counts+=1
print(counts)
#check the reshape(6,12,-1)
cosphi_adj_cos.reshape(6,12,-1)[:13]


In [ ]:
if fileSCL and fileSCR is not None:
    #SUM of S enantiomers two different helicities
    # fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for phi ordered the scannig order of axis has to be inverted
    for i,ax in zip(range(72),axes.T.ravel()) :
        #old adding up
        ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCLCR_phi[i],0.3,0.6).T, levels=5)
        ax.tick_params(labelsize=12)
        # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCLCR_phi_smth_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
#SUM of R enantiomers two different helicities
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    #old adding up
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCLCR_phi[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_RCLCR_phi_smth_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#TOTAL sum of enantiomers and helicities smoothed
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_totordered_phi[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_total_phi_smth_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#TOTAL sum of enantiomers and helicities
#fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    ax.contourf(phiM,cosM,MFPAD_totordered_phi[i].T)
    ax.tick_params(labelsize=12)
    ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_total_phi_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    for i,ax in zip(range(72),axes.ravel()) :
        ax.contourf(phiM,cosM,MFPAD_SCR_cos[i].T)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, cosphi_adj_cos[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCR_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #NOTE IT IS NEEDED TO ADD A COLORBAR! newxis are creaed in overlaygraph according to the fig size!
    #colorbar like this is squeezing in the axis, find a way to have it as an object!
    # fugi = reload(fugi)
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
    # aspect=(phiM.max()/cosM.max())

    for i,ax in zip(range(72),axes.ravel()):
        cs=ax.contourf(phiM,cosM,MFPAD_SCL_cos[i].T)
        ax.tick_params(labelsize=12)
        # ax.set_aspect(aspect)
        #ax.text(0, 0, cosphi_adj_cos[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)#, title="MFPAD SCL")
    # fig.colorbar(cs, ax=axes.ravel().tolist())
    fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCL_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for phi ordered the scannig order of axis has to be inverted
    for i,ax in zip(range(72),axes.T.ravel()) :
        ax.contourf(phiM,cosM,MFPAD_SCR_phi[i].T)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCR_phi_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for phi ordered the scannig order of axis has to be inverted
    for i,ax in zip(range(72),axes.T.ravel()) :
        ax.contourf(phiM,cosM,MFPAD_SCL_phi[i].T)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCL_phi_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
#fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.ravel()) :
    ax.imshow(fugi.smoothgauss(MFPAD_RCR_cos[i],0.3,0.6).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi_adj_cos[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_RCR_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_phi[i].T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_RCR_phi_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    ax.contourf(phiM,cosM,MFPAD_RCL_phi[i].T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_RCL_phi_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

#NOTE for phi ordered the scannig order of axis has to be inverted
for i,ax in zip(range(72),axes.T.ravel()) :
    ax.contourf(phiM,cosM,MFPAD_RCL_cos[i].T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_RCL_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show()

if fileSCL and fileSCR is not None:

    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for phi ordered the scannig order of axis has to be inverted
    for i,ax in zip(range(72),axes.ravel()) :
        ax.contourf(phiM,cosM,MFPAD_SCRtest[i].T)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    # fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCL_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for phi ordered the scannig order of axis has to be inverted
    for i,ax in zip(range(72),axes.ravel()) :
        ax.contourf(phiM,cosM,MFPAD_SCR[i].T)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)

    fugi.overlaygraph(fig)
    # fig.savefig("../PYTHON_graphs/OUTPUTS/MFPAD_"+CH+"_"+en+"_SCL_cos_exp.png", figsize=(50, 24), dpi=144, transparent=False)
    plt.show()

In [ ]:
if fileSCL and fileSCR is not None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    cs=ax.contourf(phiM,cosM,MFPAD_SCRtest_sum, cmap=cmap_temp, levels=20)
    ax.set_title("SCR tot MFPAD")

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('phi [DEG]')
    ax.set_ylabel('cos(theta) [adm]')
    cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
    cbar.set_ticklabels(["min","max"])
    cbar.set_ticks([MFPAD_SCRtest_sum.min(),MFPAD_SCRtest_sum.max()])
    cbar.ax.set_ylabel('normalised counts')
    ax.set_aspect(aspect)

    fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_singleNONORM_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
if fileSCL and fileSCR is not None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    cs=ax.contourf(phiM,cosM,fugi.normalise_matrix(MFPAD_SCL_sum,normtype=2).T, cmap=cmap_temp, levels=20)
    ax.set_title("SCR tot MFPAD n2 "+CH+"_"+en)

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('phi [DEG]')
    ax.set_ylabel('cos(theta) [adm]')
    cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
    cbar.set_ticklabels(["min","max"])
    cbar.set_ticks([fugi.normalise_matrix(MFPAD_SCL_sum,normtype=2).min(),fugi.normalise_matrix(MFPAD_SCL_sum,normtype=2).max()])
    cbar.ax.set_ylabel('normalised counts')
    ax.set_aspect(aspect)

    fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_single_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
if fileSCL and fileSCR is not None:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=20)
    ax.set_title("SCR tot MFPAD n2 "+CH+"_"+en)

    aspect=(phiM.max()/cosM.max())
    ax.set_xlabel('phi [DEG]')
    ax.set_ylabel('cos(theta) [adm]')
    cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
    cbar.set_ticklabels(["min","max"])
    cbar.set_ticks([MFPAD_tot_sum.min(),MFPAD_tot_sum.max()])
    cbar.ax.set_ylabel('normalised counts')
    ax.set_aspect(aspect)

    fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_single_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(phiM,cosM,MFPAD_tottest, cmap=cmap_temp, levels=20)
# cs=ax.imshow(MFPAD_tottest, cmap=cmap_temp, origin="lower")
ax.set_title("total MFPAD KIlian norm "+CH+"_"+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
cbar.set_ticks([MFPAD_tottest.min(),MFPAD_tottest.max()])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/total_MFPAD_kilian_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

#a try with polotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

endrange=[]

for MFPAD in MFPAD_RCR_cos:
    endrange.append(np.amax(fugi.smoothgauss(MFPAD,0.3,0.6)))
endrange=np.amax(endrange)


fig = go.Figure()
fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=fugi.smoothgauss(MFPAD_RCR_cos[j+i*6],0.3,0.6).T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=endrange, size=endrange/10)), row=j+1, col=i+1)
    
fig.update_layout(
    title=dict(
        text="MFPAD smoothed cos sorted" ,
        #y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top"),
    #coloraxis=dict(colorscale=Seismic_r),
    #showlegend=False,
    autosize=True,
    width=1000,
    #height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
if fileSCL and fileSCR is not None:
    #this can be all added togheter to create the total sum
    fig,ax = plt.subplots(2,2, figsize=(18, 15), sharex=True, sharey=True)
    st = fig.suptitle("MFPAD RS "+CH+"_"+en, fontsize="x-large")
    aspect=(phiM.max()/cosM.max())

    cs1=ax[0][0].contourf(phiM,cosM,MFPAD_RCR_sum.T,cmap=cmap_temp)
    ax[0][0].set_title("RCR")
    ax[0][0].set_ylabel('cos(theta) [adm]')
    # fig.colorbar(cs1, extend='both', shrink=0.9, ax=ax[0][0])
    ax[0][0].set_aspect(aspect)

    cs2=ax[0][1].contourf(phiM,cosM,MFPAD_SCR_flip_sum.T,cmap=cmap_temp)
    ax[0][1].set_title("SCR_flip")
    # fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax[0][1])
    ax[0][1].set_aspect(aspect)

    cs3=ax[1][0].contourf(phiM,cosM,MFPAD_RCL_sum.T,cmap=cmap_temp)
    ax[1][0].set_title("RCL")
    ax[1][0].set_xlabel('phi [DEG]')
    ax[1][0].set_ylabel('cos(theta) [adm]')
    # fig.colorbar(cs3,extend='both', shrink=0.9, ax=ax[1][0])
    ax[1][0].set_aspect(aspect)

    cs4=ax[1][1].contourf(phiM,cosM,MFPAD_SCL_flip_sum.T,cmap=cmap_temp)
    ax[1][1].set_title("SCL_flip")
    ax[1][1].set_xlabel('phi [DEG]')
    # fig.colorbar(cs4,extend='both', shrink=0.9, ax=ax[1][1])
    ax[1][1].set_aspect(aspect)

    st.set_y(0.95)
    fig.savefig("../PYTHON_graphs/OUTPUTS/total_MFPAD_sum_RS_test_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
if fileSCL and fileSCR is None:
    #this can be all added togheter to create the total sum
    fig,ax = plt.subplots(1,2, figsize=(15, 15), sharex=True, sharey=True)
    st = fig.suptitle("MFPAD "+CH+"_"+en, fontsize="x-large")
    aspect=(phiM.max()/cosM.max())
    
    cs1=ax[0].contourf(phiM,cosM,MFPAD_RCR_sum.T,cmap=cmap_temp)
    ax[0].set_title("RCR")
    ax[0].set_ylabel('cos(theta) [adm]')
    ax[0].set_xlabel('phi [DEG]')
    # fig.colorbar(cs1, extend='both', shrink=0.9, ax=ax[0][0])
    ax[0].set_aspect(aspect)
    
    cs2=ax[1].contourf(phiM,cosM,MFPAD_RCL_sum.T,cmap=cmap_temp)
    ax[1].set_title("SCR_flip")
    ax[1].set_ylabel('cos(theta) [adm]')
    ax[1].set_xlabel('phi [DEG]')
    # fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax[0][1])
    ax[1].set_aspect(aspect)
    
    fig.savefig("../PYTHON_graphs/OUTPUTS/total_MFPAD_sum_R_test_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
ax.set_title("MFPAD single sum "+CH+"_"+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
cbar.set_ticks([MFPAD_tot_sum.min(),MFPAD_tot_sum.max()])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/total_MFPAD_singlesum_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
if fileSCL and fileSCR is not None:
    #cos(theta) seems inverted rispect to contour
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=("MFPAD tot", "MFPAD SCLCR", "MFPAD RCLCR"))

    # fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2,     end=0.2, size=0.04)), 1, 1
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_tot_sum.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_SCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis3"), 1, 3)

    fig.update_layout(
        coloraxis1=dict(colorscale=cmap_temp_go,
                        colorbar=dict(x=0.335, xanchor='right', thickness=10)),
        coloraxis2=dict(colorscale=cmap_temp_go,
                        colorbar=dict(x=0.648, xanchor='left', thickness=10)),
        coloraxis3=dict(colorscale=cmap_temp_go,
                        colorbar=dict(x=1.0, xanchor='left', thickness=10)),
        # showlegend=False,
        autosize=False,
        width=1550,
        height=450,
        margin=dict(l=5,r=5,b=25,t=25)
        )

    fig.show()
else:

    #cos(theta) seems inverted rispect to contour
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=("MFPAD tot", "MFPAD SCLCR", "MFPAD RCLCR"))

    # fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2,     end=0.2, size=0.04)), 1, 1
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_tot_sum.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
    fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCLCR_sum.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)

    fig.update_layout(
        coloraxis1=dict(colorscale=cmap_temp_go,
                        colorbar=dict(x=0.525, xanchor='right', thickness=10)),
        coloraxis2=dict(colorscale=cmap_temp_go,
                        colorbar=dict(x=1.05, xanchor='left', thickness=10)),
        # showlegend=False,
        autosize=False,
        width=950,
        height=450,
        margin=dict(l=5,r=5,b=25,t=25)
        )

    fig.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(phiM,cosM,mPECD.T, cmap=cmap_temp, levels=10)
ax.set_title("contrast map "+CH+"_"+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/contrastmap_singlesum_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)